In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.datasets import make_regression
import time

# Create synthetic data for training
X_train, y_train = make_regression(n_samples=1000, n_features=20, random_state=42)
X_train_df = pd.DataFrame(X_train)  # Training with DataFrame, as you mentioned

# Train model
model = LGBMRegressor(n_estimators=50, random_state=42, verbose=-1)
model.fit(X_train_df, y_train)

# Create test input in different memory orders
X_test = np.array(X_train[:1000])  # Base array for testing

# C-contiguous (default)
X_c = np.ascontiguousarray(X_test)

# Fortran-contiguous
X_f = np.asfortranarray(X_test)

# Non-contiguous (e.g., a transpose slice)
X_nc = X_test.T[:,:]  # shape changes to (20, 10) but let's fix it by transposing back
X_nc = X_nc.T  # Back to (10, 20), still non-contiguous

def benchmark_predict(X, model, n_runs=10):
    # Warm-up
    model.predict(X)
    start = time.time()
    for _ in range(n_runs):
        model.predict(X)
    end = time.time()
    avg_time = (end - start) / n_runs
    print(f"Predict with array shape {X.shape} and memory order "
          f"C_CONTIGUOUS={X.flags['C_CONTIGUOUS']}, F_CONTIGUOUS={X.flags['F_CONTIGUOUS']}: "
          f"{avg_time*1000:.3f} ms per run")

print("Benchmarking prediction performance:\n")

benchmark_predict(X_c, model)
benchmark_predict(X_f, model)
benchmark_predict(X_nc, model)


Benchmarking prediction performance:

Predict with array shape (1000, 20) and memory order C_CONTIGUOUS=True, F_CONTIGUOUS=False: 2.024 ms per run
Predict with array shape (1000, 20) and memory order C_CONTIGUOUS=False, F_CONTIGUOUS=True: 2.248 ms per run
Predict with array shape (1000, 20) and memory order C_CONTIGUOUS=True, F_CONTIGUOUS=False: 2.110 ms per run


/home/joaquin/miniconda3/envs/skforecast_benchmark/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/joaquin/miniconda3/envs/skforecast_benchmark/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/joaquin/miniconda3/envs/skforecast_benchmark/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/joaquin/miniconda3/envs/skforecast_benchmark/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/joaquin/miniconda3/envs/skforecast_benchmark/lib/python3.12/site-packages/skle

In [13]:
import pandas as pd
import numpy as np
import time

# Create a large Series with some NaNs
n = 10_000_000
data = np.random.rand(n)
# data[1_000_000] = np.nan  # insert NaNs sparsely
s = pd.Series(data)
